Import libraries

In [2]:
from google.colab import drive 
import pandas as pd
import os

import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
import time
!pip install datasketch
from datasketch import MinHash, MinHashLSH

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 KB 3.0 MB/s eta 0:00:00


Connect with google drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Read csv files and save to dataframe

In [4]:
imdb_sample=pd.read_csv('/content/drive/My Drive/DataAnalytics/Datasets/Dataset_2/imdb_sample.csv')
imdb_test_without_labels=pd.read_csv('/content/drive/My Drive/DataAnalytics/Datasets/Dataset_2/imdb_test_without_labels.csv')
imdb_train=pd.read_csv('/content/drive/My Drive/DataAnalytics/Datasets/Dataset_2/imdb_train.csv')

train = imdb_train
test = imdb_test_without_labels
test.loc[:, 'sentiment']=imdb_sample['sentiment']

Cleanning datasets

In [5]:
def clean_text(review):
  # Tokenize the review text
  translator = str.maketrans('', '', string.punctuation)
  review = review.translate(translator)

  review_tokens = nltk.word_tokenize(review)

  # Remove stop words and apply stemming
  stop_words = set(stopwords.words("english"))
  stemmer = PorterStemmer()
  processed_tokens = [stemmer.stem(token) for token in review_tokens if token not in stop_words]
  
  processed_tokens = ' '.join(processed_tokens)
  return processed_tokens


train['review'] = (train['review']).apply(clean_text)
test['review'] = test['review'].apply(clean_text)

Create if not exist Results CSV file

In [ ]:
filename = '/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv'
if not os.path.exists(filename):
    results = pd.DataFrame(columns=["Method", "BuildTime", "QueryTime", "TotalTime", "fraction of the true K most similar documents", "Parameters"])
    results.to_csv(filename, index=False)

Brute force with jaccard similarity

In [ ]:
results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output//Output_2/results.csv')
results = results[results.Method != 'Brute-Force-Jaccard']
results.to_csv('/content/drive/My Drive/DataAnalytics/Output//Output_2/results.csv', index=False)

test_similars = []
build_time=0
query_time=0

startTime = time.time()
for index,testvalue in enumerate(test['review']):
    if index%1000==0:
      print(index)
    q1 = set(testvalue.split())
    similarities = []

    for index2,trainvalue in enumerate(train['review']):

        q2 = set(trainvalue.split())
        jaccard=len(q1.intersection(q2))/len(q1.union(q2))   
        similarities.append((index2,jaccard))

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    similarities=similarities[:15]
    test_similars.append(similarities)

print(test_similars)
query_time=time.time() - startTime
print('Query took {} seconds'.format(query_time))

listofsimilars=[]
listofsimilars2=[]
for i in test_similars:
  temp_sim=[]
  temp_sim2=[]
  result=''
  for s in i:
    temp_sim.append(s[0])
    temp_sim2.append(s[1])
  result = ''.join('{} '.format(num) for num in temp_sim)
  result2 = ''.join('{} '.format(num) for num in temp_sim2)
  listofsimilars.append(result)
  listofsimilars2.append(result2)

df = pd.DataFrame((listofsimilars), columns=["index_train"])
df['similarity']=listofsimilars2
df.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/bruteforce_jaccard.csv', index=False)

results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv')
row = ["Brute-Force-Jaccard", 0, query_time,build_time+query_time, "100%", " "]
results.loc[len(results)] = row

results.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv', index=False)
results

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,Method,BuildTime,QueryTime,TotalTime,fraction of the true K most similar documents,Parameters
0,Brute-Force-Jaccard,0,13882.040314,13882.040314,100%,


Faction which calculate the fraction between two lists of lists

In [8]:
def calculate_fraction(true_neighbors, reported_neighbors):
        true_neighbors_set = set(true_neighbors[:15])
        reported_neighbors_set = set(reported_neighbors[:15])
        common_neighbors = true_neighbors_set & reported_neighbors_set
        fraction = len(common_neighbors) / 15
        return fraction

def total_calculate_fraction(list1,list2):

    total_fraction = 0
    for true_neighbors, reported_neighbors in zip(list1, list2):
        fraction = calculate_fraction(true_neighbors, reported_neighbors)
        total_fraction += fraction

    average_fraction = total_fraction / len(list2)    

    return average_fraction*100

LSH-JACCARD

In [ ]:
results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv')
results = results[results.Method != 'LSH-Jaccard']
results.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv', index=False)

thresholds=[0.4,0.6,0.8]
params = [16,32,64]
for thresh in thresholds:
    for param in params:

        build_time=0
        query_time=0

        questionsList = []

        print('With Param: ',param,' & threshold: ',thresh)
        print('Started index building')
        startTime = time.time()

        for question in train['review']:
            question = set(question.split())
            m = MinHash(num_perm=param)
            for d in question:
                m.update(d.encode('utf8'))
            
            questionsList.append(m)

        # Create LSH index
        lsh = MinHashLSH(threshold=thresh,num_perm=param)

        counter = 0

        for m in questionsList:
            lsh.insert(str(counter), m)
            counter += 1

        build_time = time.time() - startTime
        print('Index building took {} seconds'.format(build_time)) 


        # Query test reviews
        test_similars=[]
        similarities = []
        startTime = time.time()

        for question in test['review']:
            question = set(question.split())
            m = MinHash(num_perm=param)
            for d in question:
                m.update(d.encode('utf8'))

            similarities = []
            result = lsh.query(m)
            for r in result:
                sim=m.jaccard(questionsList[int(r)])

                similarities.append((r,sim))
            similarities.sort(key=lambda x: x[1], reverse=True)
            similarities = similarities[:15]
            
            test_similars.append(similarities)
        query_time=time.time() - startTime

        print('Query took {} seconds'.format(query_time))

        listofsimilars=[]
        listofsimilars2=[]
        for i in test_similars:
          temp_sim=[]
          temp_sim2=[]
          result=''
          for s in i:
            temp_sim.append(s[0])
            temp_sim2.append(s[1])
          result = ''.join('{} '.format(num) for num in temp_sim)
          result2 = ''.join('{} '.format(num) for num in temp_sim2)
          listofsimilars.append(result)
          listofsimilars2.append(result2)

        df = pd.DataFrame((listofsimilars), columns=["index_train"])
        df['similarity']=listofsimilars2
        df.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/lsh_jaccard.csv', index=True)

        bruteforce_jaccard = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/bruteforce_jaccard.csv')
        lsh_jaccard = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/lsh_jaccard.csv')

        lsh_jaccard['index_train'] = lsh_jaccard['index_train'].apply(lambda x: '' if pd.isnull(x) else x)
        list1 = [list(map(int, item.strip().split())) for item in bruteforce_jaccard['index_train'].tolist()]
        list2 = [list(map(int, item.strip().split())) for item in lsh_jaccard['index_train'].tolist()]

        average_fraction=total_calculate_fraction(list1,list2)


        str_average_fraction=str(average_fraction)
        row = ["LSH-Jaccard", build_time, query_time,build_time+query_time, str_average_fraction+"%", "Perm="+str(param)+", threshold="+str(thresh)+""]

        results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv')
        results.loc[len(results)] = row

        results.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv', index=False)
        print(results)

With Param:  16  & threshold:  0.4
Started index building
Index building took 65.50174641609192 seconds
Query took 98.56537342071533 seconds
                Method  BuildTime     QueryTime     TotalTime  \
0  Brute-Force-Jaccard   0.000000  13882.040314  13882.040314   
1          LSH-Jaccard  65.501746     98.565373    164.067120   

  fraction of the true K most similar documents              Parameters  
0                                          100%                          
1                           1.9546666666666237%  Perm=16, threshold=0.4  
With Param:  32  & threshold:  0.4
Started index building
Index building took 71.58089852333069 seconds
Query took 21.38439965248108 seconds
                Method  BuildTime     QueryTime     TotalTime  \
0  Brute-Force-Jaccard   0.000000  13882.040314  13882.040314   
1          LSH-Jaccard  65.501746     98.565373    164.067120   
2          LSH-Jaccard  71.580899     21.384400     92.965298   

  fraction of the true K most similar d

In [9]:
results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output//Output_2/results.csv')
results = results[results.Method != 'LSH-Cosine']
results.to_csv('/content/drive/My Drive/DataAnalytics/Output//Output_2/results.csv', index=False)

class HashTable:
    def __init__(self, hash_size, inp_dimensions,threshold):
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.threshold=threshold
        self.hash_table = dict()
        self.projections = np.random.randn(self.hash_size, inp_dimensions)
        
    def generate_hash(self, inp_vector):
        bools = (np.dot(inp_vector, self.projections.T) > self.threshold).astype('float')
        return ''.join(bools.astype('str'))

    def __setitem__(self, inp_vec, label):
        hash_value = self.generate_hash(inp_vec)
        self.hash_table[hash_value] = self.hash_table\
            .get(hash_value, list()) + [label]
        
    def __getitem__(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])


thresholds=[0.4,0.6,0.8]
hashsizes = [5,10,100]
for thresh in thresholds:
    for hashsize in hashsizes:

        build_time=0
        query_time=0
        # Create the CountVectorizer object
        vectorizer = CountVectorizer()
        train_vectors = vectorizer.fit_transform(train['review'])
        test_vectors = vectorizer.transform(test['review'])

        hash_table = HashTable(hash_size=hashsize, inp_dimensions=train_vectors.shape[1],threshold=thresh)

        import time

        startTime = time.time()

        print('With Param: ',hashsize,' & threshold: ',thresh)
        print('Started index building')
        # Add the train reviews to the hash table
        for i, vec in enumerate(train_vectors):
            vec = vec.toarray()[0]

            hash_table[vec] = i

        build_time = time.time() - startTime
        print('Index building took {} seconds'.format(build_time)) 
        startTime = time.time()

        # Query the test reviews
        test_similars = []
        for i, vec in enumerate(test_vectors):
            # if i%1000==0:
            #   print(i)
            vec = vec.toarray()[0]

            similars = hash_table[vec]

            if len(similars) == 0:
                sorted_similars = []
            else:
                similars_vectors = train_vectors[similars]
                test_vec = test_vectors[i].toarray()[0]
                test_vec = test_vec.reshape(1, -1)
                cosine_similarities = cosine_similarity(test_vec, similars_vectors).flatten()
                similars_indices_and_cosine = [(index, cosine) for index, cosine in zip(similars, cosine_similarities)]
                sorted_similars = sorted(similars_indices_and_cosine, key=lambda x: x[1], reverse=True)[:15]
                threshold = thresh
                sorted_similars = [(index, cosine) for index, cosine in sorted_similars if cosine > threshold]

            test_similars.append(sorted_similars)

        query_time=time.time() - startTime
        print('Query took {} seconds'.format(query_time))

        listofsimilars=[]
        listofsimilars2=[]
        for i in test_similars:
          temp_sim=[]
          temp_sim2=[]
          result=''
          for s in i:
            temp_sim.append(s[0])
            temp_sim2.append(s[1])
          result = ''.join('{} '.format(num) for num in temp_sim)
          result2 = ''.join('{} '.format(num) for num in temp_sim2)
          listofsimilars.append(result)
          listofsimilars2.append(result2)
        df = pd.DataFrame((listofsimilars), columns=["index_train"])
        df['similarity']=listofsimilars2

        df.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/lsh_cosine.csv', index=True)
        bruteforce_jaccard = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/bruteforce_jaccard.csv')
        lsh_cosine = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/lsh_cosine.csv')
        # print(lsh_cosine['index_train'])
        lsh_cosine['index_train'] = lsh_cosine['index_train'].apply(lambda x: '' if pd.isnull(x) else x)
        list1 = [list(map(int, item.strip().split())) for item in bruteforce_jaccard['index_train'].tolist()]
        list2 = [list(map(int, item.strip().split())) for item in lsh_cosine['index_train'].tolist()]

        average_fraction=total_calculate_fraction(list1,list2)

        str_average_fraction=str(average_fraction)
        row = ["LSH-Cosine", build_time, query_time,build_time+query_time, str_average_fraction+"%","Hash_size="+str(hashsize)+", threshold="+str(thresh)+""]
  
        results = pd.read_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv')
        results.loc[len(results)] = row

        results.to_csv('/content/drive/My Drive/DataAnalytics/Output/Output_2/results.csv', index=False)
        print(results)

With Param:  5  & threshold:  0.4
Started index building
Index building took 33.662925481796265 seconds
Query took 229.17850494384766 seconds
                 Method  BuildTime     QueryTime     TotalTime  \
0   Brute-Force-Jaccard   0.000000  13882.040314  13882.040314   
1           LSH-Jaccard  65.501746     98.565373    164.067120   
2           LSH-Jaccard  71.580899     21.384400     92.965298   
3           LSH-Jaccard  85.270581     21.800694    107.071275   
4           LSH-Jaccard  64.751784     16.828859     81.580644   
5           LSH-Jaccard  80.929151     23.045134    103.974285   
6           LSH-Jaccard  85.263049     21.234895    106.497944   
7           LSH-Jaccard  64.064251     16.327624     80.391875   
8           LSH-Jaccard  71.698497     18.851691     90.550188   
9           LSH-Jaccard  80.913693     21.508701    102.422394   
10           LSH-Cosine  33.662925    229.178505    262.841430   

   fraction of the true K most similar documents                 

In [10]:
results

,Method,BuildTime,QueryTime,TotalTime,fraction of the true K most similar documents,Parameters
0,Brute-Force-Jaccard,0.000000,13882.040314,13882.040314,100%,
1,LSH-Jaccard,65.501746,98.565373,164.067120,1.9546666666666237%,"Perm=16, threshold=0.4"
2,LSH-Jaccard,71.580899,21.384400,92.965298,1.3846666666666514%,"Perm=32, threshold=0.4"
3,LSH-Jaccard,85.270581,21.800694,107.071275,0.4780000000000053%,"Perm=64, threshold=0.4"
4,LSH-Jaccard,64.751784,16.828859,81.580644,0.2533333333333327%,"Perm=16, threshold=0.6"
5,LSH-Jaccard,80.929151,23.045134,103.974285,0.14866666666666634%,"Perm=32, threshold=0.6"
6,LSH-Jaccard,85.263049,21.234895,106.497944,0.11799999999999979%,"Perm=64, threshold=0.6"
7,LSH-Jaccard,64.064251,16.327624,80.391875,0.1066666666666665%,"Perm=16, threshold=0.8"
8,LSH-Jaccard,71.698497,18.851691,90.550188,0.1066666666666665%,"Perm=32, threshold=0.8"
9,LSH-Jaccard,80.913693,21.508701,102.422394,0.10733333333333317%,"Perm=64, threshold=0.8"
